In [7]:
import importlib

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from apollo.settings import LONG_SIGNAL, NO_SIGNAL, SHORT_SIGNAL


In [8]:
# Manually reloading python module such that
# jupyter reflects changes without kernel restart

import apollo.api.yahoo_api_connector as yac
import apollo.calculations.linear_regression_channel as lrc
from apollo.utils.common import to_default_date_string

importlib.reload(yac)
importlib.reload(lrc);

In [9]:
ticker = "SPY"
start_date = "2024-01-01"
end_date = "2024-03-01"

api_connector = yac.YahooApiConnector(ticker, start_date, end_date)
dataframe = api_connector.request_or_read_prices()

dataframe;

In [10]:
WINDOW_SIZE = 10
CHANNEL_STANDARD_DEVIATION_SPREAD = 1.0

lrc_calculator = lrc.LinearRegressionChannelCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
    channel_sd_spread=CHANNEL_STANDARD_DEVIATION_SPREAD,
)

lrc_calculator.calculate_linear_regression_channel()

dataframe.dropna(inplace=True)

dataframe["signal"] = NO_SIGNAL

long = (
    (dataframe["adj close"] <= dataframe["l_bound"]) &
    (dataframe["slope"] <= dataframe["prev_slope"])
)
dataframe.loc[long, "signal"] = LONG_SIGNAL

short = (
    (dataframe["adj close"] >= dataframe["u_bound"]) &
    (dataframe["slope"] >= dataframe["prev_slope"])
)
dataframe.loc[short, "signal"] = SHORT_SIGNAL

dataframe

,ticker,open,high,low,close,adj close,volume,slope,prev_slope,l_bound,u_bound,signal
date,,,,,,,,,,,,
2024-01-17,SPY,471.820007,472.790009,469.869995,472.290009,470.827118,68843900,0.826399,0.883918,471.711368,478.652517,1
2024-01-18,SPY,474.010010,477.059998,472.420013,476.489990,475.014099,91856200,0.745076,0.826399,472.264734,478.902470,0
2024-01-19,SPY,477.649994,482.720001,476.540009,482.429993,480.935699,110733300,0.793350,0.745076,473.815733,480.806547,-1
2024-01-22,SPY,484.010010,485.220001,482.779999,483.450012,481.952576,75844900,0.756558,0.793350,475.335490,482.052041,0
2024-01-23,SPY,484.010010,485.109985,482.890015,484.859985,483.358185,49945300,1.052729,0.756558,477.014961,485.083755,0
2024-01-24,SPY,487.809998,488.769989,484.880005,485.390015,483.886566,81765000,1.205892,1.052729,478.506968,487.265082,0
2024-01-25,SPY,487.579987,488.309998,485.390015,488.029999,486.518372,72525000,1.541637,1.205892,480.489523,490.591128,0
2024-01-26,SPY,487.589996,489.119995,486.540009,487.410004,485.900299,76641600,1.691112,1.541637,482.002356,492.628726,0
2024-01-29,SPY,487.730011,491.420013,487.170013,491.269989,489.748322,61322800,1.938766,1.691112,484.038665,495.730263,0


In [11]:
fig = make_subplots(rows=1, cols=1, specs=[[{}]], vertical_spacing=0)

# Plot closing prices
fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["adj close"],
        line={"color":"blue", "width":1},
        name="Adj close",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["l_bound"],
        line={"color":"red", "width":1},
        name="Lower Bound",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["u_bound"],
        line={"color":"red", "width":1},
        name="Upper Bound",
    ),
    row=1,
    col=1,
)

fig.update_layout(
    title={"text": f"{ticker}", "x": 0.5},
    height=650,
    plot_bgcolor="#EFF5F8",
)

fig.update_yaxes(visible=False, secondary_y=True)

In [12]:
prime_value = "close"
trace_value = "signal"

x = dataframe.index.to_numpy()

y1 = dataframe[prime_value].to_numpy()
y2 = dataframe[trace_value].to_numpy()

# Create the first trace with the primary y-axis
trace1 = go.Scatter(x=x, y=y1, name=prime_value)

# Create the second trace with the secondary y-axis
trace2 = go.Scatter(x=x, y=y2, name=f"{trace_value}", yaxis="y2")

# Plot title
title = (
    f"{ticker}:"
    f" {to_default_date_string(dataframe.index.to_numpy()[0])}"
    f" - {to_default_date_string(dataframe.index.to_numpy()[-1])}"
)

# Create the layout with two y-axes
layout = go.Layout(
    title=title,
    yaxis={},
    yaxis2={"overlaying": "y", "side": "right"},
    height=650,
)

# Create the figure and add traces to it
fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.update_xaxes(
    showspikes=True,
    spikemode="across",
    spikecolor="black",
    spikethickness=0.5,
)